In [1]:
import gradio as gr
from PIL import Image

from openai import OpenAI
import base64
import requests
from IPython.display import display
from IPython.display import Image as ip_image
from PIL import Image
import json
import aspose.pdf as ap

# OpenAI API Key
api_key = "$"


In [13]:

def modifications(image): #takes file as input
    return image #modifies image / output


def greet(question): #replace this with the function that can answer the question
    return "Did you ask " + question + "?" #returns the answer to the question


#%% Image from Path

# Function to display image image from file
def display_image_file(image):
    myImage = Image.open(image)
    return myImage.show()

# Function to encode the image from file
def encode_image_file(image):
    with open(image, "rb") as image_file:
        display_image_file(image)
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
# image_path = "C:\\Users\Maanas\OneDrive - Imperial College London\Blackboard\Research Computing\Project\\test notes.jpg"

# Getting the base64 string
# base64_image = encode_image_file(image)



#%% GPT 4V text to image - normal instructions
def GPT4V_function(image):

    base64_image = encode_image_file(image)
    
    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4-vision-preview",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": "What’s written in these notes? Please translate handwriting into text precisely, and write them into Latex so I can directly copy and paste into Latex. Only return the latex code, nothing else."
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/png;base64,{base64_image}"
              }
            }
          ]
        }
      ],
      "max_tokens": 1000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    print(response.json())


    # print((response.json()['choices'][0]['message']['content']))

    #assign response to variable and clean

    tex_code = response.json()['choices'][0]['message']['content'].replace('```latex', '').replace('```', '')
    # print(tex_code)



    #save tex file

    f = open("extracted_text.tex","w+")
    f.write(tex_code)
    f.write("\n")
    f.write("\n")
    f.close()



    #https://blog.aspose.com/pdf/convert-latex-to-pdf-in-python/#:~:text=to%20PDF%20Converter-,Python%20LaTeX%20to%20PDF%20Converter,to%20PDF%20in%20this%20article

    # Convert Latex to PDF
    # Create TeXLoadOptions class object
    options = ap.TeXLoadOptions()

    # Create a Document class object
    document = ap.Document("extracted_text.tex" , options)

    # Convert Latex to PDF
#     document.save("extracted_text.pdf")
    document.save("extracted_text.pdf")
    return 'extracted_text.pdf'


In [14]:
image = "C:\\Users\Maanas\OneDrive - Imperial College London\Blackboard\Research Computing\Project\\test2.png"
# "C:\Users\Maanas\OneDrive - Imperial College London\Blackboard\Research Computing\Project\test2.png"
GPT4V_function(image)


{'id': 'chatcmpl-92m6HtJLeNN4EnFBsxQybiNRh8d6w', 'object': 'chat.completion', 'created': 1710448581, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 1152, 'completion_tokens': 410, 'total_tokens': 1562}, 'choices': [{'message': {'role': 'assistant', 'content': '```latex\n\\documentclass{article}\n\\usepackage[utf8]{inputenc}\n\\usepackage{amsmath}\n\n\\begin{document}\n\n\\section*{XII-PHYSICS}\n\n\\subsection*{Electrostatics}\n\\subsubsection*{1. Unit 2, 3 and 5 marks Questions}\n\n\\begin{enumerate}\n    \\item What do you mean by "electrically neutral" of an object?\n    \\begin{itemize}\n        \\item An object have zero net charge is said to be electrically neutral.\n    \\end{itemize}\n    \n    \\item What is mean by triboelectric charging?\n    \\begin{itemize}\n        \\item Charging the objects through rubbing is called triboelectric charging.\n    \\end{itemize}\n    \n    \\item Define "conservation of charge".\n    \\begin{itemize}\n        \\item The to

'extracted_text.pdf'

In [15]:
def display_pdf():
    return GPT4V_function()


css = """
h1 {
    text-align: center;
    display:block;
}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown(""" # Digitialise It! """)

    with gr.Row("Upload your File"):
        input=gr.Image()
        output=gr.File()
        upload_btn=gr.Button("Upload")
        upload_btn.click(fn=display_pdf,inputs=input, outputs=output)
    with gr.Row("Ask a Question"):
        question = gr.Textbox(label="Question") #textbox for question
        answer = gr.Textbox(label="Answer") #textbox for answer
        enter_btn = gr.Button("Enter") #enter button 
        enter_btn.click(fn=greet,inputs=question,outputs=answer) #displays answer to question when enter is clicked

demo.launch() 

C:\Users\Maanas\anaconda3\lib\site-packages\gradio\utils.py:857: UserWarning: Expected 0 arguments for function <function display_pdf at 0x000001AF4E6E29D0>, received 1.
  warnings.warn(
C:\Users\Maanas\anaconda3\lib\site-packages\gradio\utils.py:865: UserWarning: Expected maximum 0 arguments for function <function display_pdf at 0x000001AF4E6E29D0>, received 1.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
